# Web Scraping and Introductory Data Analysis
// purpose of assintment


// summary of what we're going to do

In [1]:
%pip install -r ./../requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.by import By

In [18]:
class Scrip_Etheruim:
    url = "https://etherscan.io/txs"

    def __init__(self):
        self.driver = webdriver.Chrome()
        self.df = pd.DataFrame(columns=['tnx_hash', 'method', 'block', 'date', 'from', 'to', 'value', 'tnx_fee'])

    def __del__(self):
        self.driver.quit()

    def _get_text(self, element):
        return element.find('a').get('href').split('/')[-1] 

    def _get_data(self, element):
        tnx_hash = element[1].text.strip()
        method = element[2].text.strip()
        block = element[3].text.strip()
        date = element[4].text.strip()
        _from = self._get_text(element[7])
        to = self._get_text(element[9])
        value = element[10].text.strip()
        tnx_fee = element[11].text.strip()
        data = [tnx_hash, method, block, date, _from, to, value, tnx_fee]
        return pd.Series(data, index=['tnx_hash', 'method', 'block', 'date', 'from', 'to', 'value', 'tnx_fee']), block


    def _extract_data_from_html(self, html_content):
        soup = BeautifulSoup(html_content, "html.parser")
        rows = soup.find_all("tr")
        block_number = 0
        for row in rows:
            cells = row.find_all("td")
            series, block = self._get_data(cells)
            block_number = max(block_number, block)
            self.df = pd.concat([self.df, pd.DataFrame([series])], ignore_index=True)

        return block_number

            
    def _extract_data_from_url(self):
        self.driver.get(self.url)
        table_body = self.driver.find_element(By.CSS_SELECTOR, "tbody.align-middle.text-nowrap")
        html_content = table_body.get_attribute("outerHTML")
        df = self._extract_data_from_html(html_content)
        time.sleep(0.5)

